In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%pip install statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose, DecomposeResult
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from scipy.stats import ttest_ind
# suppress all warnings
import warnings
warnings.filterwarnings("ignore")


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
cases = pd.read_csv('ILINet.csv', skiprows=1)
cases.head()

,REGION TYPE,REGION,YEAR,WEEK,% WEIGHTED ILI,%UNWEIGHTED ILI,AGE 0-4,AGE 25-49,AGE 25-64,AGE 5-24,AGE 50-64,AGE 65,ILITOTAL,NUM. OF PROVIDERS,TOTAL PATIENTS
0,National,X,1997,40,1.10148,1.21686,179,X,157,205,X,29,570,192,46842
1,National,X,1997,41,1.20007,1.28064,199,X,151,242,X,23,615,191,48023
2,National,X,1997,42,1.37876,1.23906,228,X,153,266,X,34,681,219,54961
3,National,X,1997,43,1.19920,1.14473,188,X,193,236,X,36,653,213,57044
4,National,X,1997,44,1.65618,1.26112,217,X,162,280,X,41,700,213,55506


In [ ]:
cases = cases[['YEAR', 'WEEK', 'ILITOTAL']]
cases

,YEAR,WEEK,ILITOTAL
0,1997,40,570
1,1997,41,615
2,1997,42,681
3,1997,43,653
4,1997,44,700
...,...,...,...
1358,2023,41,56896
1359,2023,42,61850
1360,2023,43,66922
1361,2023,44,71117


In [ ]:
cases = cases.loc[cases['WEEK'] != 53]

In [ ]:
cases['DATE'] = cases.apply(lambda row: pd.to_datetime(f'{int(row["YEAR"])}-{int(row["WEEK"])}-1', format='%G-%V-%u'), axis=1)
cases

,YEAR,WEEK,ILITOTAL,DATE
0,1997,40,570,1997-09-29
1,1997,41,615,1997-10-06
2,1997,42,681,1997-10-13
3,1997,43,653,1997-10-20
4,1997,44,700,1997-10-27
...,...,...,...,...
1358,2023,41,56896,2023-10-09
1359,2023,42,61850,2023-10-16
1360,2023,43,66922,2023-10-23
1361,2023,44,71117,2023-10-30


In [ ]:
cases['angle'] = (cases['WEEK'] - 1) * (2.0 * np.pi / 52)  

# Calculate sine and cosine and add them as new columns
cases['sin_weekly'] = np.sin(cases['angle'])
cases['cos_weekly'] = np.cos(cases['angle'])

# Drop the intermediate 'angle' column if not needed
cases = cases.drop(['angle', 'YEAR', 'DATE', 'WEEK'], axis=1)


In [ ]:

# cases['angle_week'] = (cases['WEEK'] - 1) * (2.0 * np.pi / 52)
# cases['angle_year'] = (cases['YEAR'] - 1) * (2.0 * np.pi / 26)

# # Combine week and year information
# cases['combined_angle'] = cases['angle_week'] + cases['angle_year']

# # Calculate sine and cosine based on combined angle
# cases['sin_combined'] = np.sin(cases['combined_angle'])
# cases['cos_combined'] = np.cos(cases['combined_angle'])

# # Drop intermediate columns if not needed
# cases = cases.drop(['WEEK', 'YEAR', 'angle_week', 'angle_year', 'combined_angle', 'DATE'], axis=1)


In [ ]:
cases

,ILITOTAL,sin_weekly,cos_weekly
0,570,-1.000000,-1.836970e-16
1,615,-0.992709,1.205367e-01
2,681,-0.970942,2.393157e-01
3,653,-0.935016,3.546049e-01
4,700,-0.885456,4.647232e-01
...,...,...,...
1358,56896,-0.992709,1.205367e-01
1359,61850,-0.970942,2.393157e-01
1360,66922,-0.935016,3.546049e-01
1361,71117,-0.885456,4.647232e-01


In [ ]:
train = cases[cases.index <= 1304]
test = cases[(cases.index >= 1305) & (cases.index <= 1356)]
prediction_test = cases[cases.index >= 1357]

In [ ]:
train

,ILITOTAL,sin_weekly,cos_weekly
0,570,-1.000000,-1.836970e-16
1,615,-0.992709,1.205367e-01
2,681,-0.970942,2.393157e-01
3,653,-0.935016,3.546049e-01
4,700,-0.885456,4.647232e-01
...,...,...,...
1300,37529,-0.822984,-5.680647e-01
1301,38334,-0.885456,-4.647232e-01
1302,39760,-0.935016,-3.546049e-01
1303,44666,-0.970942,-2.393157e-01


In [ ]:
test

,ILITOTAL,sin_weekly,cos_weekly
1305,57318,-1.000000e+00,-1.836970e-16
1306,63388,-9.927089e-01,1.205367e-01
1307,75615,-9.709418e-01,2.393157e-01
1308,109496,-9.350162e-01,3.546049e-01
1309,139247,-8.854560e-01,4.647232e-01
1310,140055,-8.229839e-01,5.680647e-01
1311,137419,-7.485107e-01,6.631227e-01
1312,164936,-6.631227e-01,7.485107e-01
1313,185673,-5.680647e-01,8.229839e-01
1314,167248,-4.647232e-01,8.854560e-01


In [ ]:
prediction_test

,ILITOTAL,sin_weekly,cos_weekly
1357,55806,-1.000000,-1.836970e-16
1358,56896,-0.992709,1.205367e-01
1359,61850,-0.970942,2.393157e-01
1360,66922,-0.935016,3.546049e-01
1361,71117,-0.885456,4.647232e-01
1362,82769,-0.822984,5.680647e-01


In [ ]:
X_train = train.drop(columns=['ILITOTAL'])
y_train = train['ILITOTAL']

In [ ]:
%pip install xgboost


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score


cv_splits = 5

xgb_model = XGBRegressor()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [20, 50, 100, 150, 200],  
    'max_depth': [1, 3, 5, 7, 10],           
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
}

# Create a TimeSeriesSplit cross-validator
tscv = TimeSeriesSplit(n_splits=cv_splits)

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='neg_mean_squared_error',
                           cv=tscv, verbose=1, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and corresponding MSE
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Mean Squared Error:", -grid_search.best_score_)

# Access the best model directly
best_xgb_model = grid_search.best_estimator_

# If you want to cross-validate the best model on the entire dataset, you can use cross_val_score
# This can be helpful to get a sense of the model's performance on the entire time series
cv_results = cross_val_score(best_xgb_model, X_train, y_train, scoring='neg_mean_squared_error', cv=tscv)
print("Cross-validated Mean Squared Error on the entire dataset:", np.mean(-cv_results))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
Best Mean Squared Error: 288701987.077124
Cross-validated Mean Squared Error on the entire dataset: 288701987.077124


In [ ]:
from sklearn.linear_model import LinearRegression

# Create a linear regression model
model = LinearRegression()

# Train the model on the training data
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
X_test = test.drop(columns=['ILITOTAL'])
y_test = test['ILITOTAL']

In [ ]:
from sklearn.metrics import mean_squared_error

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')

Mean Squared Error: 4303640190.832371


In [ ]:
cases = pd.read_csv('ILINet.csv', skiprows=1)
cases.head()

,REGION TYPE,REGION,YEAR,WEEK,% WEIGHTED ILI,%UNWEIGHTED ILI,AGE 0-4,AGE 25-49,AGE 25-64,AGE 5-24,AGE 50-64,AGE 65,ILITOTAL,NUM. OF PROVIDERS,TOTAL PATIENTS
0,National,X,1997,40,1.10148,1.21686,179,X,157,205,X,29,570,192,46842
1,National,X,1997,41,1.20007,1.28064,199,X,151,242,X,23,615,191,48023
2,National,X,1997,42,1.37876,1.23906,228,X,153,266,X,34,681,219,54961
3,National,X,1997,43,1.19920,1.14473,188,X,193,236,X,36,653,213,57044
4,National,X,1997,44,1.65618,1.26112,217,X,162,280,X,41,700,213,55506


In [ ]:
cases_2019_present = cases[cases['YEAR'] >= 2019]
cases_2019_present = cases_2019_present.iloc[39:].reset_index(drop=True)
cases_2019_present

,REGION TYPE,REGION,YEAR,WEEK,% WEIGHTED ILI,%UNWEIGHTED ILI,AGE 0-4,AGE 25-49,AGE 25-64,AGE 5-24,AGE 50-64,AGE 65,ILITOTAL,NUM. OF PROVIDERS,TOTAL PATIENTS
0,National,X,2019,40,1.49064,1.50415,6218,4827,X,7658,1764,1448,21915,2896,1456967
1,National,X,2019,41,1.59269,1.59823,6550,5147,X,7638,2015,1603,22953,2924,1436150
2,National,X,2019,42,1.72517,1.73838,7273,5548,X,8277,2110,1649,24857,2950,1429897
3,National,X,2019,43,1.83428,1.86050,7892,6080,X,9349,2305,1793,27419,2980,1473740
4,National,X,2019,44,2.04049,2.01425,8658,6143,X,10068,2279,1762,28910,2989,1435277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,National,X,2023,41,2.29191,2.29444,14308,13873,X,18291,5397,5027,56896,4111,2479729
211,National,X,2023,42,2.50619,2.49517,15618,14328,X,19997,5779,6128,61850,4096,2478788
212,National,X,2023,43,2.69580,2.65943,16855,15342,X,22080,6020,6625,66922,4111,2516401
213,National,X,2023,44,2.96143,2.92928,17570,16397,X,24303,6261,6586,71117,4047,2427801


In [ ]:
cases_2019_present = cases_2019_present[['YEAR', 'WEEK', 'ILITOTAL']]
cases_2019_present

,YEAR,WEEK,ILITOTAL
0,2019,40,21915
1,2019,41,22953
2,2019,42,24857
3,2019,43,27419
4,2019,44,28910
...,...,...,...
210,2023,41,56896
211,2023,42,61850
212,2023,43,66922
213,2023,44,71117


In [ ]:
cases_2019_present = cases_2019_present.loc[cases_2019_present['WEEK'] != 53]

In [ ]:
cases_2019_present['DATE'] = cases_2019_present.apply(lambda row: pd.to_datetime(f'{int(row["YEAR"])}-{int(row["WEEK"])}-1', format='%G-%V-%u'), axis=1)
cases_2019_present

,YEAR,WEEK,ILITOTAL,DATE,angle,sin_weekly,cos_weekly
0,2019,40,21915,2019-09-30,4.712389,-1.000000,-1.836970e-16
1,2019,41,22953,2019-10-07,4.833219,-0.992709,1.205367e-01
2,2019,42,24857,2019-10-14,4.954050,-0.970942,2.393157e-01
3,2019,43,27419,2019-10-21,5.074880,-0.935016,3.546049e-01
4,2019,44,28910,2019-10-28,5.195711,-0.885456,4.647232e-01
...,...,...,...,...,...,...,...
210,2023,41,56896,2023-10-09,4.833219,-0.992709,1.205367e-01
211,2023,42,61850,2023-10-16,4.954050,-0.970942,2.393157e-01
212,2023,43,66922,2023-10-23,5.074880,-0.935016,3.546049e-01
213,2023,44,71117,2023-10-30,5.195711,-0.885456,4.647232e-01


In [ ]:
cases_2019_present['angle'] = (cases_2019_present['WEEK'] - 1) * (2.0 * np.pi / 52)  

# Calculate sine and cosine and add them as new columns
cases_2019_present['sin_weekly'] = np.sin(cases_2019_present['angle'])
cases_2019_present['cos_weekly'] = np.cos(cases_2019_present['angle'])

# Drop the intermediate 'angle' column if not needed
cases_2019_present = cases_2019_present.drop(['angle', 'YEAR', 'DATE', 'WEEK'], axis=1)

In [ ]:
train = cases_2019_present[cases_2019_present.index <= 156]
test = cases_2019_present[(cases_2019_present.index >= 157) & (cases_2019_present.index <= 208)]
prediction_test = cases_2019_present[cases_2019_present.index >= 209]

In [ ]:
X_train = train.drop(columns=['ILITOTAL'])
y_train = train['ILITOTAL']

In [ ]:
cv_splits = 5

xgb_model = XGBRegressor()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [20, 50, 100, 150, 200],  
    'max_depth': [1, 3, 5, 7, 10],           
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
}

# Create a TimeSeriesSplit cross-validator
tscv = TimeSeriesSplit(n_splits=cv_splits)

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='neg_mean_squared_error',
                           cv=tscv, verbose=1, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and corresponding MSE
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Mean Squared Error:", -grid_search.best_score_)

# Access the best model directly
best_xgb_model = grid_search.best_estimator_

# If you want to cross-validate the best model on the entire dataset, you can use cross_val_score
# This can be helpful to get a sense of the model's performance on the entire time series
cv_results = cross_val_score(best_xgb_model, X_train, y_train, scoring='neg_mean_squared_error', cv=tscv)
print("Cross-validated Mean Squared Error on the entire dataset:", np.mean(-cv_results))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 20}
Best Mean Squared Error: 793856663.0697716
Cross-validated Mean Squared Error on the entire dataset: 793856663.0697716


In [ ]:
X_test = test.drop(columns=['ILITOTAL'])
y_test = test['ILITOTAL']

In [ ]:
y_pred = model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')

Mean Squared Error: 4303640190.832371


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c7e7b740-ad7d-41a8-9704-cb0c6a8385eb' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>